In [51]:
import pandas as pd
import numpy as np
import scipy as sp
times = pd.read_excel('TIMES.xlsx')
times.set_index('Número', inplace = True)
times.fillna('NA', inplace = True)
times['Porcentagem de Vitorias'] = pd.Series(np.random.randn(len(list(times.Sigla))), index=times.index)
times_msi = ['RNG','KZ','FW','FNC','TL','EVS','SUP','GMB','KBM','R7','KLG','DW','ASC','PGM']
times_mundial_inicio = ['EDG','INF','DW','G2','SUP','ASC','C9','DFM','KBM','GRX','GMB','KLG','AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100']
times_mundial_grupos = ['AFS','FW','PVB','RNG','VIT','GEN','KT','TL','MAD','FNC','IG','100','EDG','G2','C9','GRX']
times_mundial_quartas = ['RNG','KT','FNC','IG','EDG','G2','C9','AFS']
ACERTOS=0
JOGOS_TOTAIS = 0


In [52]:
def arrumadados(sheet,guia):
    arquivo = pd.ExcelFile(sheet)
    liga = pd.read_excel(arquivo,guia-1)
    liga = liga[3:]
    liga.fillna(0,inplace = True)
    liga.replace(0.7,0.5, inplace = True)
    return liga

vitorias= {}
jogos = {}
for sigla in list(times.Sigla):
    vitorias[sigla]=0
    jogos[sigla]=0

In [53]:
cblol1 = arrumadados('CBLOL_BRASIL.xlsx',1)
cblol2 = arrumadados('CBLOL_BRASIL.xlsx',2)
cls1 = arrumadados('CLS_LAS.xlsx',1)
cls2 = arrumadados('CLS_LAS.xlsx',2)
eu_lcs1 = arrumadados('EU_LCS_EUROPA.xlsx',1)
eu_lcs2 = arrumadados('EU_LCS_EUROPA.xlsx',2)
lck1 = arrumadados('LCK_COREIA.xlsx',1)
lck2 = arrumadados('LCK_COREIA.xlsx',2)
lcl1 = arrumadados('LCL_CIS.xlsx',1)
lcl2 = arrumadados('LCL_CIS.xlsx',2)
ljl1 = arrumadados('LJL_JAPAN.xlsx',1)
ljl2 = arrumadados('LJL_JAPAN.xlsx',2)
lln1 = arrumadados('LLN_LAN.xlsx',1)
lln2 = arrumadados('LLN_LAN.xlsx',2)
lms1 = arrumadados('LMS_TAIWAN.xlsx',1)
lms2 = arrumadados('LMS_TAIWAN.xlsx',2)
lpl1 = arrumadados('LPL_CHINA.xlsx',1)
lpl2 = arrumadados('LPL_CHINA.xlsx',2)
na_lcs1 = arrumadados('NA_LCS_EUA.xlsx',1)
na_lcs2 = arrumadados('NA_LCS_EUA.xlsx',2)
opl1 = arrumadados('OPL_OCEANIA.xlsx',1)
opl2 = arrumadados('OPL_OCEANIA.xlsx',2)
seatour1 = arrumadados('SEATOUR_SEA.xlsx',1)
seatour2 = arrumadados('SEATOUR_SEA.xlsx',2)
tcl1 = arrumadados('TCL_TURQUIA.xlsx',1)
tcl2 = arrumadados('TCL_TURQUIA.xlsx',2)
vcs1 = arrumadados('VCS_VIETNAN.xlsx',1)
vcs2 = arrumadados('VCS_VIETNAN.xlsx',2)
msi1 = arrumadados('MSI.xlsx',1)
msi2 = arrumadados('MSI.xlsx',2)
msi3 = arrumadados('MSI.xlsx',3)
msi4 = arrumadados('MSI.xlsx',4)
mundial1 = arrumadados('MUNDIAL_2018.xlsx',1)
mundial2 = arrumadados('MUNDIAL_2018.xlsx',2)
mundial3 = arrumadados('MUNDIAL_2018.xlsx',3)
mundial4 = arrumadados('MUNDIAL_2018.xlsx',4)
teste = arrumadados('testando.xlsx',1)

In [54]:
def elo(liga,maximo=1850,k=7):
    tam=int(liga.shape[1]/5)
    for semana in range(1,tam+1):
        somar = {}
        for sigla in list(times.Sigla):
            somar[sigla]=0
        for time in list(times.Sigla):
            for perdedorrr in list(liga['Perdedor'+str(semana)]):
                if perdedorrr!=0:
                    if time==perdedorrr:
                        jogos[perdedorrr]+=1
            for vencedor in list(liga['Vencedor'+str(semana)]):
                if vencedor!=0:
                    if time==vencedor:
                        jogos[vencedor]+=1
                        vitorias[vencedor]+=1
                        dados_lista = liga[liga['Vencedor'+str(semana)]==vencedor]
                        for num_perd in range(0,len(list(dados_lista['Perdedor1']))):
                            perdedor = list(dados_lista['Perdedor'+str(semana)])[num_perd]
                            peso = float(list(dados_lista['Peso'+str(semana)])[num_perd])
                            parametro_vencedor = 10**(times[times.Sigla==vencedor]['Força atualizada'].values/400)
                            parametro_perdedor = 10**(times[times.Sigla==perdedor]['Força atualizada'].values/400)
                            prob_ganhador = parametro_vencedor/(parametro_vencedor+parametro_perdedor)
                            prob_perdedor = parametro_perdedor/(parametro_vencedor+parametro_perdedor)
                            
                            if times[times.Sigla==vencedor]['Força atualizada'].values >= maximo:
                                somar[vencedor]+= k/2*(peso - prob_ganhador)
                            else:
                                somar[vencedor]+= k*(peso - prob_ganhador)
                            if times[times.Sigla==perdedor]['Força atualizada'].values >= maximo:
                                somar[perdedor]-= k/2*(prob_perdedor)
                            else:
                                somar[perdedor]-= k*(prob_perdedor)
                else:
                    pass
        for time in list(times.Sigla):
            ind = times[times.Sigla==time].index
            times.loc[ind,'Força atualizada']=times.loc[ind,'Força atualizada']+somar[time]
        



def previsao(timeA,timeB):
    parametro_timeA = 10**(times[times.Sigla==timeA]['Força atualizada'].values/400)
    parametro_timeB = 10**(times[times.Sigla==timeB]['Força atualizada'].values/400)
    prob_timeA = parametro_timeA/(parametro_timeA+parametro_timeB)
    prob_timeB = parametro_timeB/(parametro_timeA+parametro_timeB)
    print('A probabilidade de {} ganhar é de {}%'.format(timeA,prob_timeA))
    print('A probabilidade de {} ganhar é de {}%'.format(timeB,prob_timeB))
    
def por_vit():
    porcentagem_de_vitorias = {}
    for time in list(times.Sigla):
        porcentagem_de_vitorias[time] = vitorias[time]/jogos[time]
    for time in list(times.Sigla):
        times.replace(times[times.Sigla==time]['Porcentagem de Vitorias'].values,porcentagem_de_vitorias[time], inplace = True)
def elasticidade(a1,a2,b,c1,c2,regiao):
    posicao_certa = {}
    i=1
    for time in list(times[times.Região==regiao].Sigla):
        posicao_certa[time]=i
        i+=1
    
    elo(a1,b,c1)
    elo(a2,b,c2)

    quad = 0
    
    posicao_calculada = {}
    j=1
    for time in list(times[times.Região==regiao].sort_values(by=['Força atualizada'],ascending = False).round(2).Sigla):
        posicao_calculada[time]=j
        j+=1
    
    for i in list(times[times.Região==regiao].Sigla):
        quad+= (posicao_certa[i] - posicao_calculada[i])**(2)
    return quad/len(list(times[times.Região==regiao].Sigla))

def acerto(liga):
    tam = int(liga.shape[1]/5)
    global ACERTOS
    global JOGOS_TOTAIS
    for semana in range(1,tam+1):
        vencedores = list(liga['Vencedor'+str(semana)])
        perdedores = list(liga['Perdedor'+str(semana)])
        for num in range(0,len(vencedores)):
            if times[times.Sigla==vencedores[num]]['Força atualizada'].values > times[times.Sigla==perdedores[num]]['Força atualizada'].values:
                ACERTOS+=1
            else:
                pass
            JOGOS_TOTAIS+=1
                    
                    
                            
                

In [55]:
elo(msi1,1800,11)
elo(msi2,1800,11)
elo(msi3,1800,11)
elo(msi4,1800,5.5)
elo(lpl1,1800,9)
elo(lpl2,1800,9)
elo(na_lcs1,1800,5)
elo(na_lcs2,1800,2.5)
elo(lck1,1950,6)
elo(lck2,1950,3)
elo(eu_lcs1,1850,8)
elo(eu_lcs2,1850,4)
elo(lms1,1850,7)
elo(lms2,1850,14)
elo(cblol1)
elo(cblol2)
elo(cls1)
elo(cls2)
elo(lcl1)
elo(lcl2)
elo(ljl1)
elo(ljl2)
elo(lln1)
elo(lln2)
elo(opl1)
elo(opl2)
elo(seatour1)
elo(seatour2)
elo(tcl1)
elo(tcl2)
elo(vcs1)
elo(vcs2)
consulta_dic = {}
for time in list(times.Sigla):
    consulta_dic[time] = float(times[times.Sigla==time]['Força atualizada'])
    
por_vit()


In [56]:
acerto(mundial1)
acerto(mundial2)
acerto(mundial3)
acerto(mundial4)
print('A porcentagem de acerto do seu modelo no mundial foi de {:.2F}%'.format(100*ACERTOS/JOGOS_TOTAIS))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


A porcentagem de acerto do seu modelo no mundial foi de 52.98%


In [ ]:
#Funcao usada para calcular o melhor K para cada liga pelo método de mínima diferenca dos erros ao quadrado
maximos = [1800,1850,1900,1950,2000,10000]
porc = [0.5,1,1.5,2,2.5,3]
for KR in range(1,31):
    for MM in maximos:
        for KP in porc:
            
            for time in list(times.Sigla):
                ind = times[times.Sigla==time].index
                times.loc[ind,'Força atualizada'] = consulta_dic[time]


            VALOR = elasticidade(lms1,lms2,MM,KR,KP*KR,'TWHKMA')
            justica = list(times[times.Região=='TWHKMA'].sort_values(by=['Força atualizada'],ascending = False).round(2).Sigla)
            if VALOR<= 3:
                print(KR,KP*KR,MM,VALOR,justica[:5])
            else:
                 pass
                

In [9]:
times[times.Sigla.isin(times_mundial_inicio)].sort_values(by=['Força atualizada'],ascending = False).round(2)

,Sigla,Região,Força Inicial,Força atualizada,Porcentagem de Vitorias
Número,,,,,
25,FW,TWHKMA,1750,1918.47,0.76
1,KT,KR,1850,1901.27,0.67
2,GEN,KR,1850,1898.40,0.63
3,AFS,KR,1850,1881.51,0.59
11,RNG,CN,1800,1869.84,0.78
33,FNC,EU,1800,1844.47,0.64
12,IG,CN,1800,1841.83,0.90
34,VIT,EU,1800,1840.48,0.64
35,G2,EU,1800,1836.01,0.56


In [10]:
times.sort_values(by=['Força atualizada'],ascending = False).head(16).round(2)

,Sigla,Região,Força Inicial,Força atualizada,Porcentagem de Vitorias
Número,,,,,
25,FW,TWHKMA,1750,1918.47,0.76
1,KT,KR,1850,1901.27,0.67
2,GEN,KR,1850,1898.40,0.63
3,AFS,KR,1850,1881.51,0.59
11,RNG,CN,1800,1869.84,0.78
33,FNC,EU,1800,1844.47,0.64
12,IG,CN,1800,1841.83,0.90
34,VIT,EU,1800,1840.48,0.64
35,G2,EU,1800,1836.01,0.56
